In [1]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import numpy as np

from new.CustomVAE import *
from utils import calculate_model_losses

In [2]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

# no KL divergence loss
args.use_AE = True

# tensorboard
# writer = SummaryWriter()

# load data
vocab, train_loader, val_loader = build_loaders(args)

dt = train_loader.dataset

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 20000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender

Starting to read 

In [3]:
args.use_AE = False

In [4]:
for batch in tqdm(train_loader):
    #t += 1
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    z = torch.randn(objs.size(0), 64).to(objs.device)
    break

  0%|          | 0/3367 [00:00<?, ?it/s]

In [5]:
obj_to_img.shape

torch.Size([226])

In [6]:
from new.CustomVAE import TransformerEncoder

In [8]:
kwargs = {
        'vocab': dt.vocab,
        'batch_size': args.batch_size,
        'train_3d': args.train_3d,
        'decoder_cat': args.decoder_cat,
        'embedding_dim': 84,
        'gconv_mode': args.gconv_mode,
        'gconv_num_layers': args.gconv_num_layers,
        'mlp_normalization': args.mlp_normalization,
        'vec_noise_dim': args.vec_noise_dim,
        'layout_noise_dim': args.layout_noise_dim,
        'use_AE': args.use_AE
    }

In [9]:
model = TransformerEncoder(**kwargs)

In [10]:
model = model.cuda()

In [11]:
obj_vecs = model.obj_embeddings_ec(objs)
if model.use_attr:
    attr_vecs = model.attr_embedding_ec(attributes)
    obj_vecs = torch.cat([obj_vecs, attr_vecs], dim=1)
angle_vecs = model.angle_embeddings(angles)
boxes_vecs = model.box_embeddings(boxes)

In [12]:
obj_vecs = torch.cat([obj_vecs, boxes_vecs, angle_vecs], dim=1)

In [13]:
obj_vecs = obj_vecs.unsqueeze(0)

In [14]:
obj_vecs.shape

torch.Size([1, 226, 168])

In [16]:
# attention mask
obj_counts = [torch.sum(obj_to_img == i).item() for i in range(model.batch_size)]
block_list = [torch.ones((obj_counts[i],obj_counts[i])) for i in range(model.batch_size)]
attention_mask = torch.block_diag(*block_list).to(obj_vecs.device) # [BxB]

In [18]:
attention_mask[None,None,:,:].shape

torch.Size([1, 1, 226, 226])

In [20]:
# the attention mask is expand as [1 x 1 x B x B]
bert_outputs = model.bert_encoder(obj_vecs, attention_mask=attention_mask[None,None,:,:]) # [1 x B x D]

self attention torch.Size([1, 12, 226, 226]) torch.Size([1, 1, 226, 226]) 
 tensor([[[[1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.]]]], device='cuda:0')
self attention torch.Size([1, 12, 226, 226]) torch.Size([1, 1, 226, 226]) 
 tensor([[[[1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.]]]], device='cuda:0')
self attention torch.Size([1, 12, 226, 226]) torch.Size([1, 1, 226, 226]) 
 tensor([[[[1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 

In [24]:
bert_outputs = bert_outputs[0]

In [26]:
subject_linear_output = model.subject_linear(bert_outputs) # [1 x B x D']
object_linear_output = model.object_linear(bert_outputs) # [1 x B x D']

In [27]:
subject_linear_output.shape

torch.Size([1, 226, 84])

In [28]:
subject_linear_output = subject_linear_output.squeeze(0)
object_linear_output = object_linear_output.squeeze(0)


In [29]:
subject_linear_output.shape

torch.Size([226, 84])

In [31]:
score_matrix = torch.matmul(subject_linear_output, object_linear_output.transpose(0, 1)) * attention_mask

In [32]:
score_matrix.shape

torch.Size([226, 226])

In [35]:
score_matrix[0:10,0:10]

tensor([[2.0817, 2.0977, 2.4928, 3.0574, 2.9056, 1.9862, 2.0171, 2.0443, 1.8758,
         2.1376],
        [2.3692, 2.5199, 2.6113, 2.8264, 2.3392, 2.6950, 2.3713, 2.1056, 1.6135,
         2.4800],
        [3.6730, 4.0350, 4.4525, 5.4143, 3.7011, 5.1430, 4.0372, 3.5844, 2.4112,
         4.8002],
        [2.6439, 3.2576, 2.5389, 3.4189, 2.2304, 3.3003, 3.1469, 1.8072, 1.6662,
         2.9595],
        [2.6145, 2.5038, 3.0985, 3.3311, 2.6613, 2.1068, 3.0979, 2.7883, 2.3179,
         2.7465],
        [3.0530, 2.9330, 4.1122, 4.1907, 3.0697, 2.9409, 2.8209, 3.2432, 2.3394,
         2.9094],
        [4.2413, 4.8809, 3.8992, 5.9086, 4.1115, 4.5958, 3.1125, 3.3925, 2.6061,
         4.5617],
        [3.0253, 2.8248, 3.4274, 4.0565, 2.7640, 2.8270, 2.9669, 2.7714, 2.1909,
         3.1671],
        [4.8614, 5.6234, 4.8591, 6.8841, 4.0719, 4.5254, 4.1504, 3.1802, 3.6211,
         4.2249],
        [3.4033, 3.6181, 3.4449, 3.4908, 3.1506, 2.8512, 3.6625, 3.5046, 2.9415,
         2.7295]], device='c

In [ ]:
 model.use_attr

In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel(BertConfig())


In [ ]:
inputs

In [ ]:
inputs = tokenizer(["Hello, my dog is cute","what is it"], return_tensors="pt",padding=True)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [ ]:
attr_vecs.shape

In [ ]:
inputs

In [ ]:
inputs = tokenizer(["Hello, my dog is cute","what is it"], return_tensors="pt",padding=True)

In [ ]:
inputs